In [1]:
import os
import pickle
import argparse

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

In [11]:
# Global parameters


# temporal resolution: "W" for weekly, or "M" for monthly
freq = "W"

# The number of decision trees in Random Forest ensemble
n_trees = 10 # recommended: 100

# to subset only June, July, August, and September: True
# to consider all available TSL measurements: False
subset_jjas = False

# to consider only mean temperature: True
# to consider mean, min, max temperatures: False
only_t2m_mean = True

if freq == "M":
    freq_prefix = "monthly"
elif freq == "W":
    freq_prefix = "weekly"

if subset_jjas:
    subset_prefix = "JJAS"
else:
    subset_prefix = "full"

# Dir for results
results_path = f"../results/{freq_prefix}_{subset_prefix}_domain/"
results_path_models = f"../results/{freq_prefix}_{subset_prefix}_domain/trained_models/"
results_path_simulations = f"../results/{freq_prefix}_{subset_prefix}_domain/simulations/"

if not os.path.exists(results_path):
    os.mkdir(results_path)
if not os.path.exists(results_path_models):
    os.mkdir(results_path_models)
if not os.path.exists(results_path_simulations):
    os.mkdir(results_path_simulations)

In [2]:
# Defining sources of data

# RGI IDs
glacier_ids = np.load("../data/misc/glacier_IDs.npy", allow_pickle=True)

# TSLs
tsl_store = pd.read_pickle("../data/tsl/TSLs.pkl")

# Meteo forcing
meteo_path = "../data/meteo/agg-weekly/"

In [4]:
# Physiographic attributes
static_features = pd.read_csv("../data/misc/rgi60_Asia.csv")

In [5]:
# HELPERS

In [6]:
def read_tsl(rgi_id, store=tsl_store):
    
    tsl = tsl_store[tsl_store['RGI_ID'] == rgi_id].copy()
    
    tsl.index = pd.to_datetime(tsl['LS_DATE'])
    
    tsl = pd.DataFrame(tsl['TSL_m'])    
    
    return tsl


def read_meteo(rgi_id, path=meteo_path):
    
    meteo = pd.read_hdf(f"{meteo_path}{rgi_id}.h5")
    meteo.index = pd.to_datetime(meteo['date'])
    meteo = meteo.drop(['date', 'wind_dir_mean_labels'], axis=1)
    
    return meteo


def create_features(dataframe, back_to=12):
    
    # convert circular wind_dir_mean 
    # to two components of cos() and sin()
    # source: https://stats.stackexchange.com/questions/336613/
    # regression-using-circular-variable-hour-from-023-as-predictor
    
    # copy for safety
    df = dataframe.copy()
    
    # create cos() and sin() components
    df["wind_dir_mean_cos"] = np.cos(np.deg2rad(df["wind_dir_mean"]))
    df["wind_dir_mean_sin"] = np.sin(np.deg2rad(df["wind_dir_mean"]))
    
    # drop "wind_dir_mean"
    df = df.drop(["wind_dir_mean"], axis=1)
    
    # make shifts and rolling means
    cols = df.columns
    for col in cols:
        for shift in range(1, back_to+1, 1):
            df["{}-{}".format(col, shift)] = df[col].shift(shift).values
        for rol in range(1, back_to+1, 1):
            df["{}rol-{}".format(col, rol)] = df[col].rolling(window=rol).mean().values
    
    # delete NaNs
    df = df.dropna()
       
    return df


def datasets_construction(rgi_id, freq, subset_jjas, only_t2m_mean):
    
    # get raw TSL measurements
    tsl = read_tsl(rgi_id)
    
    # resample to specific frequency
    tsl_resample = tsl.resample(freq).mean()
    
    # get raw ERA5-Land forcing
    meteo = read_meteo(rgi_id)
    
    # resample to specific frequency
    meteo_resample = pd.DataFrame({'t2m_min'  : meteo['t2m_min'].resample(freq).min(), 
                                   't2m_max'  : meteo['t2m_max'].resample(freq).max(), 
                                   't2m_mean' : meteo['t2m_mean'].resample(freq).mean(), 
                                   'd2m'      : meteo['d2m'].resample(freq).mean(),
                                   
                                   'sp'       : meteo['sp'].resample(freq).mean(),
                                   
                                   'tp'       : meteo['tp'].resample(freq).sum(),
                                   'sf'       : meteo['sf'].resample(freq).sum(),
                                   
                                   'ssrd_mean': meteo['ssrd_mean'].resample(freq).sum(), 
                                   'strd_mean': meteo['strd_mean'].resample(freq).sum(),
                                   
                                    
                                   'wind_mean': meteo['wind_mean'].resample(freq).mean(), 
                                   'wind_dir_mean': meteo['wind_dir_mean'].resample(freq).mean(),
                                   })
    
    if only_t2m_mean:
        meteo_resample = meteo_resample.drop(['t2m_min', 't2m_max'], axis=1)
    
    core_meteo_features = meteo_resample.columns.tolist()
    
    # enrich meteo features
    if freq == "M":
        meteo_enrich = create_features(meteo_resample, back_to=12)
    elif freq == "W":
        meteo_enrich = create_features(meteo_resample, back_to=48) #12 months back considering 4 weeks in each month
    
    
    # meteo for the entire period
    # for model evaluation
    meteo_full = meteo_enrich.dropna()
    
    # merge datasets
    dataset = pd.concat([tsl_resample, meteo_enrich], axis=1)
    
    # drop NaNs
    dataset = dataset.dropna()
    
    if subset_jjas:
        dataset = dataset[(dataset.index.month == 6) | (dataset.index.month == 7) | 
                          (dataset.index.month == 8) | (dataset.index.month == 9)]
    
    return dataset, meteo_full, core_meteo_features

In [34]:
def basin_wise(rgi_id, freq, subset_jjas, only_t2m_mean):
    
    data, _, core_features = datasets_construction(rgi_id, freq, subset_jjas, only_t2m_mean)
        
    static_features_slice = static_features[static_features["RGIId"]==rgi_id].copy()
    
    static_features_slice = static_features_slice[['CenLon', 'CenLat', 'Area', 'Zmin', 'Zmax', 'Zmed', 
                                                   'Slope', 'Aspect', 'Lmax']].copy()
    
    for c in static_features_slice.columns:
        data[c] = static_features_slice[c].values[0] 
        
    core_features = core_features + static_features_slice.columns.tolist()
    
    return data, core_features

In [27]:
def tiny_prep(df):
       
    df_X = df.drop(["TSL_m"], axis=1)
    df_y = df["TSL_m"]
    
    return df_X, df_y

In [33]:
def calculate_importances(imp_instance, core_features):
    
    cols = imp_instance.columns.tolist()
    
    core_feature_cols = {key: [i for i in cols if key in i] for key in core_features}
    
    """
    # temperature-based
    t2m_min_cols = [i for i in cols if "t2m_min" in i]
    t2m_max_cols = [i for i in cols if "t2m_max" in i]
    t2m_mean_cols = [i for i in cols if "t2m_mean" in i]
    
    # precipitation-based
    tp_cols = [i for i in cols if "tp" in i]
    sf_cols = [i for i in cols if "sf" in i]
    
    # surface solar radiation downwards
    ssrd_cols = [i for i in cols if "ssrd" in i]
    
    # surface thermal radiation downwards
    strd_cols = [i for i in cols if "strd" in i]
    
    # wind-based
    wind_max_cols = [i for i in cols if "wind_max" in i]
    wind_mean_cols = [i for i in cols if "wind_mean" in i]
    wind_dir_mean_cols = [i for i in cols if "wind_dir_mean" in i]
    
    # total cloud cover
    tcc_cols = [i for i in cols if "tcc" in i]
    """
    var_importances = []
    
    """
    for var in [t2m_min_cols, 
                t2m_max_cols, 
                t2m_mean_cols, 
                tp_cols, 
                sf_cols,
                ssrd_cols, 
                strd_cols, 
                wind_max_cols, 
                wind_mean_cols, 
                wind_dir_mean_cols,
                tcc_cols]:
    """    
    for var in core_feature_cols.values():
        
        var_importances.append(imp_instance[var].sum(axis=0).sum())
        
    var_importances = np.array(var_importances)
    
    var_importances = var_importances / var_importances.sum()
    
    return var_importances

In [35]:
d, core_features = basin_wise("RGI60-13.00014", freq, subset_jjas, only_t2m_mean)

In [36]:
d.head()

,TSL_m,t2m_mean,d2m,sp,tp,sf,ssrd_mean,strd_mean,wind_mean,wind_dir_mean_cos,...,wind_dir_mean_sinrol-48,CenLon,CenLat,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax
1989-08-06,5568.0,269.083557,264.517609,51168.113281,0.163268,0.139850,237.087082,120.036118,1.876868,-0.987759,...,-0.265044,78.0681,35.5749,0.649,5559,5953,5745,30.2,51,615
1991-04-07,5571.0,253.201050,249.759186,50769.199219,0.013774,0.013236,233.367905,90.768661,2.591038,-0.972666,...,-0.455834,78.0681,35.5749,0.649,5559,5953,5745,30.2,51,615
1991-05-12,5571.0,258.191711,254.732483,50987.050781,0.048553,0.047331,262.753418,96.882195,2.161129,-0.875503,...,-0.447250,78.0681,35.5749,0.649,5559,5953,5745,30.2,51,615
1991-09-01,5579.0,270.421265,262.456482,51504.546875,0.111383,0.062296,227.447083,107.406357,1.327983,-0.847400,...,-0.418638,78.0681,35.5749,0.649,5559,5953,5745,30.2,51,615
1992-02-09,5583.0,239.504517,236.343079,49792.695312,0.057578,0.057304,143.100708,63.163300,2.399455,-0.992695,...,-0.490425,78.0681,35.5749,0.649,5559,5953,5745,30.2,51,615


In [37]:
core_features

['t2m_mean',
 'd2m',
 'sp',
 'tp',
 'sf',
 'ssrd_mean',
 'strd_mean',
 'wind_mean',
 'wind_dir_mean',
 'CenLon',
 'CenLat',
 'Area',
 'Zmin',
 'Zmax',
 'Zmed',
 'Slope',
 'Aspect',
 'Lmax']

In [44]:
print("Data collections in progress...")

data_file = os.path.join(results_path, "data.csv")
    
for i, idx in enumerate(glacier_ids[0:10]):

    chunk, _ = basin_wise(idx, freq, subset_jjas, only_t2m_mean)
    
    if i == 0:
        chunk.to_csv(data_file, mode="a", index=False, header=True)
    else:
        chunk.to_csv(data_file, mode="a", index=False, header=False)

Data collections in progress...


In [45]:
print("Data preparation has been finished.")

Data preparation has been finished.


In [46]:
data = pd.read_csv(data_file)

In [47]:
X_df, y_df = tiny_prep(data)
X, y = X_df.to_numpy(), y_df.to_numpy()

In [48]:
X.shape, y.shape

((1754, 979), (1754,))

In [49]:
print("Modeling in progress...")

kf = KFold(n_splits=5, shuffle=True, random_state=42)

importances = []
train_scores = []
test_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # fit model
    m = RandomForestRegressor(n_jobs=-1, n_estimators=n_trees, random_state=42).fit(X_train, y_train)
    
    # save model
    pickle.dump(m, open(os.path.join(results_path_models, f"RF_{i}.pkl") , "wb"))
    
    # make predictions for the test set
    y_test_pred = m.predict(X_test)
    
    # save predictions for the test test
    sim_df = pd.DataFrame({"TSL_obs": y_test, "TSL_sim": y_test_pred})
    sim_df.to_csv(os.path.join(results_path_simulations, f"RF_{i}.csv"))    
    
    # compute scores
    train_score = m.score(X_train, y_train)
    test_score = m.score(X_test, y_test)
    
    # add scores to holders
    train_scores.append(train_score)
    test_scores.append(test_score)
    
    print(f"Fold {i+1}: train {train_score}, test {test_score}")
    
    # calculate feature imortances
    fi = m.feature_importances_
    
    # convert importances to dataframe
    fi_df = pd.DataFrame({0: fi}, index=X_df.columns).T

    # collect
    importances.append(fi_df)

Fold 1: train 0.9863566873706161, test 0.9394068956284933
Fold 2: train 0.9879554403103405, test 0.8883329656471487
Fold 3: train 0.9861676415086235, test 0.919839167231866
Fold 4: train 0.9848355596227674, test 0.9191529317823577
Fold 5: train 0.9847864254405201, test 0.9044761637054436


In [54]:
FI = calculate_importances(pd.concat(importances, axis=0, ignore_index=True), core_features)

In [56]:
FI = pd.DataFrame(FI.reshape(1,-1), columns=core_features, index=['domain'])

In [58]:
FI.to_csv(os.path.join(results_path, "importances.csv"))

In [59]:
scores = pd.DataFrame({"Train": train_scores, "Test": test_scores})

In [61]:
scores.to_csv(os.path.join(results_path, "scores.csv"))

In [62]:
print("Modeling has been finished.")

Modeling has been finished.
